In [0]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [code]
import os
import cv2
from PIL import Image
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, BatchNormalization, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline

# %% [code]
labels = os.listdir('../input/flowers-recognition/flowers/')
print(labels)

# %% [code]
X = []
Z = []
IMG_SIZE = 128
FLOWER_DAISY_DIR = "../input/flowers-recognition/flowers/daisy"
FLOWER_DANDELION_DIR = "../input/flowers-recognition/flowers/dandelion"
FLOWER_ROSE_DIR = "../input/flowers-recognition/flowers/rose"
FLOWER_SUNFLOWER_DIR = "../input/flowers-recognition/flowers/sunflower"
FLOWER_TULIP_DIR = "../input/flowers-recognition/flowers/tulip"

# %% [code]
def assign_label(img, flower_type):
    return flower_type

# %% [code]
def make_train_data(flower_type, DIR):
    for img in tqdm(os.listdir(DIR)):
        label = assign_label(img, flower_type)
        path = os.path.join(DIR, img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))

# %% [code]
make_train_data('Daisy', FLOWER_DAISY_DIR)

# %% [code]
make_train_data('Dandelion', FLOWER_DANDELION_DIR)

# %% [code]
make_train_data('Sunflower', FLOWER_SUNFLOWER_DIR)

# %% [code]
make_train_data('Tulip', FLOWER_TULIP_DIR)

# %% [code]
make_train_data('Rose', FLOWER_ROSE_DIR)

# %% [code]
fig, ax = plt.subplots(5,5)
fig.set_size_inches(12,12)
for i in range(5):
    for j in range(5):
        l = random.randint(0, len(Z))
        ax[i, j].imshow(X[l])
        ax[i, j].set_title('Flower : ' +Z[l])
        
plt.tight_layout()

# %% [code]


# %% [markdown]
# X and Y dataset.

# %% [code]
encoder = LabelEncoder()
Y = encoder.fit_transform(Z)
Y = keras.utils.to_categorical(Y, num_classes = 5)

X = np.array(X)
X = X / 255

# %% [code]
Y

# %% [markdown]
# Splitting of the image dataset.

# %% [code]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.35, random_state = 42)

# %% [code]
X_train.shape , y_train.shape , X_test.shape , y_test.shape

# %% [code]
np.random.seed(42)
random.seed(42)

# %% [markdown]
# **Keras Model.**

# %% [code]
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'same', activation = 'relu', input_shape = (128,128,3), name = 'Conv2D_1'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (3,3)))
model.add(keras.layers.SpatialDropout2D(0.2))

model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'same', activation = 'relu', name = 'Conv2D_2'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (3,3)))
model.add(keras.layers.SpatialDropout2D(0.2))

model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'same', activation = 'relu', name = 'Conv2D_3'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (3,3)))
model.add(keras.layers.SpatialDropout2D(0.2))

model.add(Conv2D(filters = 128, kernel_size = (5,5), padding = 'same', activation = 'relu', name = 'Conv2D_4'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (3,3)))
model.add(keras.layers.SpatialDropout2D(0.2))

model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(5, activation = 'softmax'))

model.compile(loss = keras.losses.CategoricalCrossentropy(),
             optimizer = keras.optimizers.Adam(learning_rate = 0.001),
             metrics = [keras.metrics.CategoricalAccuracy()])

# %% [code]
model.summary()

# %% [markdown]
# To reduce overfitting.

# %% [code]
datagen = ImageDataGenerator(
    
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range= 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False    # randomly flip images
) 

datagen.fit(X_train)

# %% [code]
model_history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 64),
                                    epochs = 50, validation_data = (X_test, y_test),
                                    verbose = 1, steps_per_epoch=X_train.shape[0] // 64)

# %% [markdown]
# Evaluation of the model.

# %% [code]
model_eva = model.evaluate(X_test, y_test)

print('Loss : ', model_eva[0])
print('Accuracy : ', model_eva[1]*100)

# %% [markdown]
# Graph regarding loss and accuracy.

# %% [code]
pd.DataFrame(model_history.history).plot(figsize = (15,10))
plt.gca()
plt.grid(True)
plt.show()

# %% [code]
accuracy = model_history.history['categorical_accuracy']
plt.plot(range(len(accuracy)), accuracy, label = 'accuracy')
plt.legend()
plt.grid(True)

# %% [code]
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.grid(True)
plt.show()

# %% [code]
y_pred = model.predict_classes(X_test)
y_pred

# %% [code]
y_test = y_test.argmax(1)
y_test

# %% [code]
y_test = y_test.reshape(-1,1)

# %% [code]
L = 8
W = 8
fig,axes = plt.subplots(L,W,figsize=(15,15))
axes = axes.ravel()

for i in np.arange(0,L*W):
    axes[i].imshow(X_test[i])
    axes[i].set_title('Prediction= {} \n True={}'.format(y_pred[i], y_test[i]))
    axes[i].axis('off')
    plt.subplots_adjust(wspace=1)

# %% [markdown]
# **THE END**

# %% [code]
